In [2]:
import pandas as pd
import requests
import folium
from urllib.parse import urlencode
from urllib.parse import quote_plus
import xml.etree.ElementTree as ET
import numpy as np

# 1. 동탄역 정차 노선 목록
> 교통카드 빅데이터 시스템

In [2]:
df_east = pd.read_csv('./GTX_동탄역_data/교통카드 빅데이터/동탄역(동측)_노선·정류장 지표(정류장별 이용량)_20240617.csv', encoding = 'euc-kr')
df_west = pd.read_csv('./GTX_동탄역_data/교통카드 빅데이터/동탄역(서측)_노선·정류장 지표(정류장별 이용량)_20240617.csv', encoding = 'euc-kr')

In [66]:
df_east

,정류장명,정류장번호,노선,기종점,일자,이용자유형,04(승차),04(하차),05(승차),05(하차),...,23(하차),00(승차),00(하차),01(승차),01(하차),02(승차),02(하차),03(승차),03(하차),Unnamed: 54
0,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,일반인,3,0,7,0,...,7,0,15,0,0,0,0,0,0,NaN
1,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,어린이,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,청소년,0,0,0,0,...,1,0,1,0,0,0,0,0,0,NaN
3,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,경로,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,동탄역(동측),55399,1000,장외리.오리골 - 장외리.오리골,2024-05,장애인,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,동탄역(동측),55399,H9,방교동일반산업단지 - 방교동일반산업단지,2024-05,장애인,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
198,동탄역(동측),55399,H9,방교동일반산업단지 - 방교동일반산업단지,2024-05,국가유공자,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
199,동탄역(동측),55399,화성똑버스01,동탄역(동측) - 동탄역(서측),2024-05,일반인,0,0,52,5,...,116,0,0,0,0,0,0,0,0,NaN
200,동탄역(동측),55399,화성똑버스01,동탄역(동측) - 동탄역(서측),2024-05,어린이,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [12]:
#동탄역(동측+서측)에 정차하는 노선 목록 확인
east = df_east['노선'].unique().tolist()
west = df_west['노선'].unique().tolist()
bus_lane = (east+west)

In [14]:
len(bus_lane)

62

In [13]:
#노선수
bus_lane

['1000',
 '17',
 '17-1',
 '19-3',
 '19-5(A)',
 '19-5(B)',
 '200',
 '203',
 '205',
 '7-1A',
 '701A',
 '709',
 '73',
 '81',
 '8202',
 '9',
 '90',
 '94',
 'G6010',
 'H1',
 'H100',
 'H101',
 'H102',
 'H11',
 'H12',
 'H15',
 'H17',
 'H18',
 'H2',
 'H20',
 'H24',
 'H4',
 'H6(A)',
 'H65',
 'H67',
 'H9',
 '화성똑버스01',
 '19-3',
 '200',
 '203',
 '205',
 '39',
 '45',
 '7-1A',
 '701A',
 '709-1',
 '9',
 '90',
 'G6010',
 'H1',
 'H100',
 'H101',
 'H102',
 'H11',
 'H12',
 'H17',
 'H18',
 'H2',
 'H24',
 'H4',
 'H6(B)',
 'H65']

# 2. 세대 현황
> 행정안전부

In [20]:
toll = pd.read_excel('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/행정안전부/행안부_202405_202405_주민등록인구및세대현황_월간.xlsx')
# 특정 행을 컬럼으로 설정하기
toll.columns = toll.iloc[1]

# 불필요한 행 제거 (첫 두 행 제거)
toll = toll[2:]

# 인덱스 리셋 (기존 인덱스는 드롭)
toll = toll.reset_index(drop=True)

toll

c:\Users\kartboy10\Documents\GTX-A-Bus-lane-optimization\.conda\lib\site-packages\openpyxl\styles\stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


1,행정기관코드,행정기관,총 거주자수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
0,4159000000,경기도 경기도 화성시,"949,720","407,372",2.33,"492,894","456,826",1.08
1,4159025300,경기도 화성시 봉담읍,"98,700","41,334",2.39,"49,906","48,794",1.02
2,4159025600,경기도 화성시 우정읍,"17,275","9,307",1.86,"9,508","7,767",1.22
3,4159025900,경기도 화성시 향남읍,"85,845","40,355",2.13,"46,143","39,702",1.16
4,4159026200,경기도 화성시 남양읍,"59,129","29,219",2.02,"32,001","27,128",1.18
5,4159031000,경기도 화성시 매송면,"6,614","3,473",1.90,"3,513","3,101",1.13
6,4159032000,경기도 화성시 비봉면,"7,656","4,609",1.66,"4,376","3,280",1.33
7,4159033000,경기도 화성시 마도면,"6,697","3,970",1.69,"4,041","2,656",1.52
8,4159034000,경기도 화성시 송산면,"10,655","5,631",1.89,"5,847","4,808",1.22
9,4159035000,경기도 화성시 서신면,"7,154","4,070",1.76,"3,974","3,180",1.25


In [35]:
toll['행정기관'].unique()

array(['경기도 경기도 화성시 ', '경기도 화성시 봉담읍', '경기도 화성시 우정읍', '경기도 화성시 향남읍',
       '경기도 화성시 남양읍', '경기도 화성시 매송면', '경기도 화성시 비봉면', '경기도 화성시 마도면',
       '경기도 화성시 송산면', '경기도 화성시 서신면', '경기도 화성시 팔탄면', '경기도 화성시 장안면',
       '경기도 화성시 양감면', '경기도 화성시 정남면', '경기도 화성시 새솔동', '경기도 화성시 진안동',
       '경기도 화성시 병점1동', '경기도 화성시 병점2동', '경기도 화성시 반월동', '경기도 화성시 기배동',
       '경기도 화성시 화산동', '경기도 화성시 동탄1동', '경기도 화성시 동탄2동', '경기도 화성시 동탄3동',
       '경기도 화성시 동탄4동', '경기도 화성시 동탄5동', '경기도 화성시 동탄6동', '경기도 화성시 동탄7동',
       '경기도 화성시 동탄8동', '경기도 화성시 동탄9동'], dtype=object)

# 3. 버스 정류장 이용량 지표
> 교통카드 빅데이터 시스템

In [19]:
df = pd.read_excel('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/교통카드 빅데이터/발생량_도착량_이용량 지표(목적통행량)_20240617.xlsx')
df

,시간대:,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,합계,Unnamed: 7,04,Unnamed: 9,...,23,Unnamed: 47,00,Unnamed: 49,01,Unnamed: 51,02,Unnamed: 53,03,Unnamed: 55
0,시도코드,시도,시군구,읍면동,이용자유형,월,발생량,도착량,발생량,도착량,...,발생량,도착량,발생량,도착량,발생량,도착량,발생량,도착량,발생량,도착량
1,합계,0,0,0,0,0,2057675,2034813,891,65,...,22646,69613,7616,33096,3470,6588,1838,1933,160,375
2,41,경기도,화성시,금곡동,경로,202405,353,410,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,국가유공자,202405,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,어린이,202405,10,10,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0,0,0,0,국가유공자,202405,104,91,0,0,...,0,4,1,3,0,0,0,0,0,0
82,0,0,0,0,어린이,202405,3513,4129,0,0,...,0,16,0,2,0,0,0,0,0,0
83,0,0,0,0,일반인,202405,136193,114799,74,10,...,1489,5006,585,2564,352,387,163,71,12,13
84,0,0,0,0,장애인,202405,676,645,0,0,...,2,10,5,7,1,0,4,1,0,0


# 4. 정차 노선수
>교통카드 빅데이터 시스템

In [21]:
df = pd.read_csv('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/교통카드 빅데이터/노선·정류장 지표(정류장별 정차 노선수)_20240617.csv', encoding = 'euc-kr')
df

,시도,시군구,읍면동,정류장번호,정류장명,노선수,정차 노선 명
0,경기도,화성시,오산동,55399,동탄역(동측),36,"1000, 17, 17-1, 19-3, 19-5(A), 19-5(B), 200, 2..."
1,경기도,화성시,오산동,55398,동탄역(서측),26,"19-3, 200, 203, 205, 39, 45, 7-1A, 701A, 709-1..."


# 5. 버스 정류장 주소 데이터
> 국토교통부_공공데이터 포털

In [24]:
df = pd.read_csv('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/공공데이터포털/(수정)경기도 화성시 버스 정류장 주소.csv')
df

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,도로명 주소
0,GGB233000459,석포3리,37.120617,126.830883,2023-10-16,36886.0,31240,경기도 화성시,경기,경기 화성시 장안면 석포리 115-1
1,GGB233000460,석포2리,37.129267,126.837217,2023-10-16,37438.0,31240,경기도 화성시,경기,경기 화성시 팔탄면 월문리 248
2,GGB233000461,석포4리,37.130050,126.839583,2023-10-16,36885.0,31240,경기도 화성시,경기,경기 화성시 장안면 석포리 426-2
3,GGB233000462,화당리,37.122017,126.864833,2023-10-16,36884.0,31240,경기도 화성시,경기,경기 화성시 팔탄면 월문리 64-2
4,GGB233000463,월문초등학교,37.117900,126.871517,2023-10-16,37437.0,31240,경기도 화성시,경기,경기 화성시 장안면 석포리 415-3
...,...,...,...,...,...,...,...,...,...,...
3021,GGB277103853,송산마도IC(경유),37.209900,126.759633,2023-10-16,0.0,31240,경기도 화성시,경기,경기 화성시 송산면 삼존리 803-1
3022,GGB277103854,송산마도IC(경유),37.209383,126.759967,2023-10-16,0.0,31240,경기도 화성시,경기,경기 화성시 송산면 삼존리 806
3023,GGB277103887,조암IC(경유),37.118633,126.818267,2023-10-16,0.0,31240,경기도 화성시,경기,경기 화성시 향남읍 행정리 472-1
3024,GGB277103888,조암IC(경유),37.118417,126.818867,2023-10-16,0.0,31240,경기도 화성시,경기,경기 화성시 우정읍 주곡리 837-39


# 6. 버스 정류장 배차 간격 데이터
> 공공데이터 포털

In [127]:
df = pd.read_excel('./GTX_동탄역_data/공공데이터포털/공공데이터포털_경기도 화성시_마을버스노선현황_20230905.xlsx')
df

,운수사,노선번호,기점,종점,출퇴근배차,평일배차,주말배차,주중상행첫차,주중상행막차,주말상행첫차,주말상행막차,주중하행첫차,주중하행막차,주말하행첫차,주말하행막차
0,금오운수,17,병점역사거리,동탄역(동측),15,15~20,25~35,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00
1,금오운수,17-1,병점역후문,동탄역,40,40~60,150~160,06:00:00,22:30:00,06:00:00,22:30:00,08:20:00,17:10:00,-,-
2,금오운수,27,병점역사거리,한림대성심병원,8,8~12,15~20,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00
3,금오운수,27-2,병점역후문,나루고교,97,97~117,-,07:43:00,16:11:00,-,-,08:33:00,17:09:00,-,-
4,다원교통주식회사,22,수현초.수현중,어천역,15,15~20,15~20,05:40:00,20:50:00,05:40:00,20:50:00,05:06:00,21:15:00,06:05:00,21:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,화성도시공사,H50-7,화성의과학대학교,사강복지회관,90,90~110,90~110,06:00:00,22:00:00,06:40:00,22:40:00,06:40:00,22:40:00,06:40:00,22:40:00
168,화성도시공사,H50-9,화성의과학대학교,화성시청서희스타힐스1단지,90,90~110,90~110,05:30:00,21:20:00,05:30:00,21:20:00,06:00:00,21:50:00,06:00:00,21:50:00
169,화성도시공사,H51,전곡항,백미리사랑방,100,100~120,100~120,06:00:00,22:00:00,06:00:00,22:00:00,06:40:00,22:40:00,06:40:00,22:40:00
170,화성도시공사,H52,전곡항,궁평유원지,110,110~140,110~140,06:00:00,21:50:00,06:00:00,21:50:00,06:45:00,22:35:00,06:45:00,22:35:00


# 배차 간격의 평균 값 계산
> 마을버스노선현황의 '출퇴근배차간격'과 동탄역을 정차하는 노선 62개에 대해서 매핑

In [62]:
# '노선번호' 컬럼을 string으로 변환
df['노선번호'] = df['노선번호'].astype(str)

# 제외할 한글 포함 데이터 제거
df = df[~df['노선번호'].str.contains(r'[ㄱ-힣]+')]

df['노선번호'].unique()

array(['17', '17-1', '27', '27-2', '22', '22-2', '50-1', '50-5A', '50-5B',
       '6-3', '6-4', '50-2', '50-2(A)', '22-4(A)', '22-4(B)', '22-5',
       '22-6', '50-8', '22-1', '20-5', '11', '11-2', '12', '13', 'H13(A)',
       'H13(B)', 'H25', '100', '100(A)', '35-1(A)', '35-1(B)', '35-2',
       '38', '38-1', '19', '19-1', '19-2', '19-3', '19-5(A)', '19-5(B)',
       '39', 'H11', 'H12', 'H16', 'H17', 'H9', '6-1', '6-2', '31', '31-1',
       '31-2', 'E1', '20-4(A)', '3-2', '333', '50', '50-3', '7-1', '709',
       '709-1', '76', '77', '78', '79', '80', '81', '9', '9-1', 'H4',
       'H5', 'H6(A)', 'H6(B)', '5', '6', 'H14(A)', 'H14(B)', 'H15', 'H19',
       'H20', '10-2', '10-3', '10-3A', '10', '10-1', '10-5', '11-1',
       '12-1', '13-1', '13-1A', '13-2', '13-3', '13-3(A)', '13-4',
       '13-4(A)', '36', '36A', '36-1', '36-2', '36-2A', '36-3', '37',
       '39A', '39-1', '39-2', 'H7', 'H7-1', 'H1', 'H10-4', 'H10-6',
       'H10-7', 'H10-8', 'H13-5', 'H13-7', 'H2', 'H50', 'H50-3', 'H5

In [63]:
#동탄역 정차 노선 번호 리스트
print(bus_lane)

['1000', '17', '17-1', '19-3', '19-5(A)', '19-5(B)', '200', '203', '205', '7-1A', '701A', '709', '73', '81', '8202', '9', '90', '94', 'G6010', 'H1', 'H100', 'H101', 'H102', 'H11', 'H12', 'H15', 'H17', 'H18', 'H2', 'H20', 'H24', 'H4', 'H6(A)', 'H65', 'H67', 'H9', '화성똑버스01', '19-3', '200', '203', '205', '39', '45', '7-1A', '701A', '709-1', '9', '90', 'G6010', 'H1', 'H100', 'H101', 'H102', 'H11', 'H12', 'H17', 'H18', 'H2', 'H24', 'H4', 'H6(B)', 'H65']


In [ ]:
# 리스트와 일치하는 노선번호만 필터링
filtered_routes = df[df['노선번호'].isin(bus_lane)]
filtered_routes = filtered_routes.reset_index(drop=True)

# 115index의 값(30~30)을 30으로 바꿔줌
filtered_routes['출퇴근배차'][16] = "30"

# Converting '출퇴근배차' to numeric
filtered_routes["출퇴근배차"] = pd.to_numeric(filtered_routes["출퇴근배차"])

# Calculating the mean
mean_value = filtered_routes["출퇴근배차"].mean()

In [67]:
filtered_routes

,운수사,노선번호,기점,종점,출퇴근배차,평일배차,주말배차,주중상행첫차,주중상행막차,주말상행첫차,주말상행막차,주중하행첫차,주중하행막차,주말하행첫차,주말하행막차
0,금오운수,17,병점역사거리,동탄역(동측),15,15~20,25~35,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00
1,금오운수,17-1,병점역후문,동탄역,40,40~60,150~160,06:00:00,22:30:00,06:00:00,22:30:00,08:20:00,17:10:00,-,-
2,산척여객,19-3,LH1단지.한양수자인,병점역사거리,40,40~50,60~70,05:15:00,21:20:00,05:15:00,21:20:00,06:00:00,22:15:00,06:00:00,22:15:00
3,산척여객,19-5(A),장지동농협,동탄역(동측),50,50~60,50~60,05:30:00,20:48:00,05:30:00,20:48:00,06:10:00,21:28:00,06:10:00,21:28:00
4,산척여객,19-5(B),장지동농협,병점역사거리,65,65~115,65~115,05:30:00,19:10:00,05:30:00,19:10:00,07:12:00,20:52:00,07:12:00,20:52:00
5,산척여객,39,동탄 물류단지,동탄역(서측),80,80~100,80~100,05:50:00,19:35:00,05:50:00,19:35:00,06:30:00,20:15:00,06:30:00,20:15:00
6,산척여객,H11,방아다리마을,현대트랜시스,20,20~30,20~30,05:30:00,21:00:00,05:30:00,21:00:00,06:35:00,22:00:00,06:35:00,22:00:00
7,산척여객,H12,장지동농협,신영통현대타운.두산위브,20,20~50,20~50,05:15:00,20:40:00,05:15:00,20:40:00,06:40:00,22:05:00,06:40:00,22:05:00
8,산척여객,H17,장지동농협,오산역,30,30~34,30~34,05:20:00,20:15:00,05:20:00,20:15:00,05:20:00,21:50:00,05:20:00,21:50:00
9,산척여객,H9,방교동일반산업단지,동탄역(동측),15,15~20,15~20,05:30:00,21:35:00,05:30:00,21:35:00,06:05:00,22:10:00,06:05:00,22:10:00


In [65]:
#출퇴근배차간격 평균 시간(분)
mean_value

np.float64(32.27272727272727)

# 7. 교통량

In [69]:
df = pd.read_excel('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/교통카드 빅데이터/발생량_도착량_이용량 지표(목적통행량)_20240617.xlsx')
df

,시간대:,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,합계,Unnamed: 7,04,Unnamed: 9,...,23,Unnamed: 47,00,Unnamed: 49,01,Unnamed: 51,02,Unnamed: 53,03,Unnamed: 55
0,시도코드,시도,시군구,읍면동,이용자유형,월,발생량,도착량,발생량,도착량,...,발생량,도착량,발생량,도착량,발생량,도착량,발생량,도착량,발생량,도착량
1,합계,0,0,0,0,0,2057675,2034813,891,65,...,22646,69613,7616,33096,3470,6588,1838,1933,160,375
2,41,경기도,화성시,금곡동,경로,202405,353,410,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,국가유공자,202405,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,어린이,202405,10,10,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,0,0,0,0,국가유공자,202405,104,91,0,0,...,0,4,1,3,0,0,0,0,0,0
82,0,0,0,0,어린이,202405,3513,4129,0,0,...,0,16,0,2,0,0,0,0,0,0
83,0,0,0,0,일반인,202405,136193,114799,74,10,...,1489,5006,585,2564,352,387,163,71,12,13
84,0,0,0,0,장애인,202405,676,645,0,0,...,2,10,5,7,1,0,4,1,0,0


# 버스 노선 최적화
> 탐욕 + 유전 알고리즘

In [3]:
#동탄역(서측)의 노선 확인
df_west = pd.read_csv('./GTX_동탄역_data/교통카드 빅데이터/동탄역(서측)_노선·정류장 지표(정류장별 이용량)_20240617.csv', encoding = 'euc-kr')
df_west.head(2)

,정류장명,정류장번호,노선,기종점,일자,이용자유형,04(승차),04(하차),05(승차),05(하차),...,23(하차),00(승차),00(하차),01(승차),01(하차),02(승차),02(하차),03(승차),03(하차),Unnamed: 54
0,동탄역(서측),55398,19-3,LH1단지.한양수자인 - LH1단지.한양수자인,2024-05,일반인,0,0,4,45,...,0,0,0,0,0,0,0,0,0,NaN
1,동탄역(서측),55398,19-3,LH1단지.한양수자인 - LH1단지.한양수자인,2024-05,어린이,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


In [6]:
df_west['노선'].unique()

array(['19-3', '200', '203', '205', '39', '45', '7-1A', '701A', '709-1',
       '9', '90', 'G6010', 'H1', 'H100', 'H101', 'H102', 'H11', 'H12',
       'H17', 'H18', 'H2', 'H24', 'H4', 'H6(B)', 'H65'], dtype=object)

In [4]:
#버스 정류장 정차 노선 수
stop = pd.read_csv('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/교통카드 빅데이터/노선·정류장 지표(정류장별 정차 노선수)_20240617.csv', encoding = 'euc-kr')
stop

,시도,시군구,읍면동,정류장번호,정류장명,노선수,정차 노선 명
0,경기도,화성시,오산동,55399,동탄역(동측),36,"1000, 17, 17-1, 19-3, 19-5(A), 19-5(B), 200, 2..."
1,경기도,화성시,오산동,55398,동탄역(서측),26,"19-3, 200, 203, 205, 39, 45, 7-1A, 701A, 709-1..."


* 25개인 이유는, '화성똑버스01'이 위의 데이터에는 포함이 안돼 있어서 그랬음

## 경기도 화성시 버스 정류소 위치 지도 시각화
> 카카오맵 api로 도로명 주소 불러온 거임.(반송동과 석우동을 sorting하려고)

In [5]:
stop_cord = pd.read_csv('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/공공데이터포털/국토교통부_(수정)경기도 화성시 버스 정류장 주소.csv', encoding='euc-kr')
stop_cord.head(2)

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,도로명 주소
0,GGB233000459,석포3리,37.120617,126.830883,2023-10-16,36886,31240,경기도 화성시,경기,경기 화성시 장안면 석포리 115-1
1,GGB233000460,석포2리,37.129267,126.837217,2023-10-16,37438,31240,경기도 화성시,경기,경기 화성시 팔탄면 월문리 248


In [21]:
#결측치
stop_cord['도로명 주소'].isnull().sum()

np.int64(0)

* 도로명 주소로 변환 못하는 결측치가 있음(21개)
  -> 결측치들은 수작업으로 위/경도 좌표로 채워넣음(네이버 지도)


In [14]:
# 동탄1동 행정구역은 반송동(일부) & 석우동임.
new_stop_cord = stop_cord[stop_cord['도로명 주소'].str.contains('반송동|석우동')]

#동탄1동의 범위를 좌표값으로 sorting
new_stop_cord = new_stop_cord[(new_stop_cord['위도'] > 37.19576) & (stop_cord['위도'] < 37.22613)]
new_stop_cord = new_stop_cord[(new_stop_cord['경도'] > 127.0594) & (stop_cord['경도'] < 127.0887)]
new_stop_cord

C:\Users\kartboy10\AppData\Local\Temp\ipykernel_10308\1641501668.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_stop_cord = new_stop_cord[(new_stop_cord['위도'] > 37.19576) & (stop_cord['위도'] < 37.22613)]
C:\Users\kartboy10\AppData\Local\Temp\ipykernel_10308\1641501668.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  new_stop_cord = new_stop_cord[(new_stop_cord['경도'] > 127.0594) & (stop_cord['경도'] < 127.0887)]


,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,도로명 주소
291,GGB233001218,메타폴리스(중),37.204267,127.067483,2023-10-16,36666,31240,경기도 화성시,경기,경기 화성시 반송동 248-1
292,GGB233001219,메타폴리스(중),37.203733,127.067517,2023-10-16,36667,31240,경기도 화성시,경기,경기 화성시 반송동 248
297,GGB233001224,다은마을(중),37.199983,127.067583,2023-10-16,36664,31240,경기도 화성시,경기,경기 화성시 반송동 180
577,GGB233001281,예당마을.롯데캐슬(중),37.210950,127.072067,2023-10-16,36672,31240,경기도 화성시,경기,경기 화성시 반송동 257
579,GGB233001283,센트럴파크뷰아파트(중),37.199433,127.064150,2023-10-16,37590,31240,경기도 화성시,경기,경기 화성시 석우동 107
792,GGB233001437,개나리공원,37.211350,127.067717,2023-10-16,37533,31240,경기도 화성시,경기,경기 화성시 반송동 109
793,GGB233001438,개나리공원,37.211700,127.067550,2023-10-16,36721,31240,경기도 화성시,경기,경기 화성시 석우동 25-9
805,GGB233001450,솔빛마을.서해.쌍용아파트,37.195817,127.069633,2023-10-16,36680,31240,경기도 화성시,경기,경기 화성시 반송동 264-3
838,GGB233001486,노작마을.한화우림,37.196183,127.077333,2023-10-16,37616,31240,경기도 화성시,경기,경기 화성시 반송동 243-4
1077,GGB233002674,화성미디어센터,37.202750,127.073933,2023-10-16,55725,31240,경기도 화성시,경기,경기 화성시 반송동 243-2


In [83]:
##저장
#new_stop_cord.to_csv('동탄1동 범위 내의 정류장 리스트.csv', index=False, encoding='utf-8-sig')

In [22]:
# 지도 생성 (초기 중심을 동탄역으로 설정)
map_center = [37.201167, 127.095111]
m = folium.Map(location=map_center, zoom_start=15, width='70%', height='70%')

# 정류소 위치에 마커 추가
for idx, row in new_stop_cord.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=f"{row['정류장명']} ({row['도로명 주소']})",
        tooltip=row['정류장명']
    ).add_to(m)

m

# 정류장별 이용량 파악(반송동, 석우동 위주)
> 경기데이터드림_노선별 버스 이용객수 데이터_API사용

In [68]:
#API 호출
def fetch_data(api_key, page_size=1000):   ### 전체 데이터 509,254건 parsing 해오는 코드
    url = "https://openapi.gg.go.kr/LINEBUSANULUSERCNT"
    params = {
        "KEY": api_key,
        "Type": "json",
        "pIndex": 1,
        "pSize": page_size
    }
    
    all_records = []
    page_index = 1
    
    while True:
        params['pIndex'] = page_index
        response = requests.get(url, params=params)
        data = response.json()
        
        if 'LINEBUSANULUSERCNT' in data:
            cnt_data = data['LINEBUSANULUSERCNT']
            if isinstance(cnt_data, list) and len(cnt_data) > 1:
                head_data = cnt_data[0]['head']
                for item in head_data:
                    if 'RESULT' in item and item['RESULT']['CODE'] == 'INFO-000':
                        records = cnt_data[1]['row']
                        all_records.extend(records)
                        break
                else:
                    print(f"API 요청에 실패했습니다. 페이지: {page_index}")
                    break
            else:
                print(f"데이터 형식이 올바르지 않습니다. 페이지: {page_index}")
                break
        else:
            print(f"데이터 형식이 올바르지 않습니다. 페이지: {page_index}")
            break
        
        # 진행 상황 출력
        print(f"페이지 {page_index} 수집 완료, 현재 데이터 개수: {len(all_records)}")
        
        page_index += 1
        
        # 모든 페이지를 다 수집했으면 종료
        if len(records) < page_size:
            break
    
    return all_records

def filter_and_sort(records): #### 전체 데이터 중에서, 경기도 화성시만 sorting 해오는 코드
    # '경기도 화성시'인 데이터만 필터링
    filtered_records = [record for record in records if record['REGION'] == '경기도 화성시']
    
    # pandas DataFrame으로 변환
    df = pd.DataFrame(filtered_records)
    
    # 정렬
    sorted_df = df.sort_values(by=['YY', 'MT'])  # YY(년도), MT(월) 기준으로 정렬
    
    return sorted_df

# API 키와 함께 데이터 가져오기
api_key = "e7d7b15de88d46068616e419e312de4d"  # 실제 인증키로 변경 필요
all_data = fetch_data(api_key)

# 데이터 필터링 및 정렬
if all_data:
    df_sorted = filter_and_sort(all_data)
    print(df_sorted)
else:
    print("데이터를 가져오는 중 오류가 발생했습니다.")


페이지 1 수집 완료, 현재 데이터 개수: 1000
페이지 2 수집 완료, 현재 데이터 개수: 2000
페이지 3 수집 완료, 현재 데이터 개수: 3000
페이지 4 수집 완료, 현재 데이터 개수: 4000
페이지 5 수집 완료, 현재 데이터 개수: 5000
페이지 6 수집 완료, 현재 데이터 개수: 6000
페이지 7 수집 완료, 현재 데이터 개수: 7000
페이지 8 수집 완료, 현재 데이터 개수: 8000
페이지 9 수집 완료, 현재 데이터 개수: 9000
페이지 10 수집 완료, 현재 데이터 개수: 10000
페이지 11 수집 완료, 현재 데이터 개수: 11000
페이지 12 수집 완료, 현재 데이터 개수: 12000
페이지 13 수집 완료, 현재 데이터 개수: 13000
페이지 14 수집 완료, 현재 데이터 개수: 14000
페이지 15 수집 완료, 현재 데이터 개수: 15000
페이지 16 수집 완료, 현재 데이터 개수: 16000
페이지 17 수집 완료, 현재 데이터 개수: 17000
페이지 18 수집 완료, 현재 데이터 개수: 18000
페이지 19 수집 완료, 현재 데이터 개수: 19000
페이지 20 수집 완료, 현재 데이터 개수: 20000
페이지 21 수집 완료, 현재 데이터 개수: 21000
페이지 22 수집 완료, 현재 데이터 개수: 22000
페이지 23 수집 완료, 현재 데이터 개수: 23000
페이지 24 수집 완료, 현재 데이터 개수: 24000
페이지 25 수집 완료, 현재 데이터 개수: 25000
페이지 26 수집 완료, 현재 데이터 개수: 26000
페이지 27 수집 완료, 현재 데이터 개수: 27000
페이지 28 수집 완료, 현재 데이터 개수: 28000
페이지 29 수집 완료, 현재 데이터 개수: 29000
페이지 30 수집 완료, 현재 데이터 개수: 30000
페이지 31 수집 완료, 현재 데이터 개수: 31000
페이지 32 수집 완료, 현재 데이터 개수: 32000
페이지 33 수집 완료, 현재 데이터 개수: 3

In [120]:
passenger = pd.read_csv('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/경기도교통정보센터/노선별 상하차 인원 데이터.csv')
passenger

,년도,월,지역,지역코드,버스노선,시점-종점 정보,시간,시간코드,주말/평일 구분,주말/평일 구분 코드,단일 승차인원,단일 하차인원,단일 총승객인원,환승 승차인원,환승 하차인원,환승 총승객인원,전체 승차인원,전체 하차인원,전체 총승객인원
0,2023,6,경기도 화성시,3124000,1007,수원대학교-잠실광역환승센터,04시~05시,4,주말,2,4,1,5,0,0,0,4,1,5
1,2023,6,경기도 화성시,3124000,1007,수원대학교-잠실광역환승센터,05시~06시,5,주말,2,32,18,50,5,2,7,37,20,57
2,2023,6,경기도 화성시,3124000,1007,수원대학교-잠실광역환승센터,06시~07시,6,주말,2,32,26,58,37,22,59,69,48,117
3,2023,6,경기도 화성시,3124000,1007,수원대학교-잠실광역환승센터,07시~08시,7,주말,2,50,46,96,43,27,70,93,73,166
4,2023,6,경기도 화성시,3124000,1007,수원대학교-잠실광역환승센터,08시~09시,8,주말,2,80,72,152,34,47,81,114,119,233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49979,2023,12,경기도 화성시,3124000,May-50,수원역환승센터(5승강장)-청요리,14시~15시,14,주말,2,4,2,6,5,2,7,9,4,13
49980,2023,12,경기도 화성시,3124000,May-50,수원역환승센터(5승강장)-청요리,15시~16시,15,주말,2,4,0,4,0,0,0,4,0,4
49981,2023,12,경기도 화성시,3124000,May-50,수원역환승센터(5승강장)-청요리,15시~16시,15,주말,2,1,3,4,1,4,5,2,7,9
49982,2023,12,경기도 화성시,3124000,May-50,수원역환승센터(5승강장)-청요리,16시~17시,16,주말,2,4,4,8,0,0,0,4,4,8


In [122]:
### 출퇴근 시간(6시~9시 및 18시~20시) + 평일 데이터로 sorting

# 필터링 조건 설정
시간_조건 = ['06시~07시', '07시~08시', '08시~09시', '18시~19시', '19시~20시']
평일_조건 = '평일'

# 조건에 맞는 데이터 필터링
filtered_passenger = passenger[(passenger['시간'].isin(시간_조건)) & (passenger['주말/평일 구분'] == 평일_조건)]

# 데이터 정렬 (원하는 기준으로 정렬, 예: 시간 순)
sorted_passenger = filtered_passenger.reset_index(drop=True)
sorted_passenger


,년도,월,지역,지역코드,버스노선,시점-종점 정보,시간,시간코드,주말/평일 구분,주말/평일 구분 코드,단일 승차인원,단일 하차인원,단일 총승객인원,환승 승차인원,환승 하차인원,환승 총승객인원,전체 승차인원,전체 하차인원,전체 총승객인원
0,2023,6,경기도 화성시,3124000,79,향남부영9단지-경기도종합사격장,07시~08시,7,평일,1,8,6,14,1,0,1,9,6,15
1,2023,6,경기도 화성시,3124000,79,향남부영9단지-경기도종합사격장,08시~09시,8,평일,1,5,6,11,1,2,3,6,8,14
2,2023,6,경기도 화성시,3124000,79,향남부영9단지-경기도종합사격장,18시~19시,18,평일,1,3,3,6,2,2,4,5,5,10
3,2023,6,경기도 화성시,3124000,27,경진여객차고지-두레자연중고등학교,07시~08시,7,평일,1,341,267,608,72,47,119,413,314,727
4,2023,6,경기도 화성시,3124000,27,경진여객차고지-두레자연중고등학교,08시~09시,8,평일,1,344,389,733,100,114,214,444,503,947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7117,2023,12,경기도 화성시,3124000,11월 02일,병점역.화성청년지원센터-서천2차아이파크,18시~19시,18,평일,1,229,258,487,163,140,303,392,398,790
7118,2023,12,경기도 화성시,3124000,11월 02일,병점역.화성청년지원센터-서천2차아이파크,19시~20시,19,평일,1,1,1,2,1,1,2,2,2,4
7119,2023,12,경기도 화성시,3124000,11월 02일,병점역.화성청년지원센터-서천2차아이파크,19시~20시,19,평일,1,130,152,282,122,137,259,252,289,541
7120,2023,12,경기도 화성시,3124000,11월 04일,발안만세시장-경진여객차고지,07시~08시,7,평일,1,7,0,7,2,0,2,9,0,9


## 정류장별 경유노선 파악

### * 그 전에 정류소ID를 매핑해야함

> 공공데이터 포털_경기도 용인시 - 버스 정류장 현황 조회 서비스_API

In [80]:
trans_stop = new_stop_cord['정류장명'].unique()
trans_stop

array(['메타폴리스(중)', '다은마을(중)', '예당마을.롯데캐슬(중)', '센트럴파크뷰아파트(중)', '개나리공원',
       '솔빛마을.서해.쌍용아파트', '노작마을.한화우림', '화성미디어센터', '라마다호텔(공항버스)',
       '동탄2동행정복지센터', '동탄1동행정복지센터.헌혈의집', '현대아이파크', '예당예원초', '롯데케슬정문',
       '미즈파크병원', '센트럴파크', '한화꿈에그린아파트', '메타폴리스', '예당마을', '신일유토빌정문',
       '전하리교회.우미제일', '센트럴포레스트정문', '능동주공단지', '동탄보건지소.우체국', '우림필유',
       '동탄고.동탄청소년문화의집', '서해그랑블아파트', '대우푸르지오.예당고등학교', '동탄119안전센터',
       '반월e편한세상', '잎새지하차도', 'IT단지(중)', '동양파라곤', '우미제일.전하리교회', '동탄홈플러스',
       '솔빛동탄경남아너스빌'], dtype=object)

In [87]:
## 정류장ID 검색##

# 예시로 주어진 코드
url = 'http://apis.data.go.kr/6410000/busstationservice/getBusStationList'

for idx, keyword in enumerate(trans_stop):
    params = {
        'serviceKey': 'LhZifyxFqTFXCd5U9nlPliJfFyenrhzHAxtL+fIivOqevEAY6Q3hB/mZEIXViYW2bJF+b2xE5ixjrxl5aC0Zcw==',
        'keyword': keyword
    }

    response = requests.get(url, params=params)

    # XML 파싱
    root = ET.fromstring(response.text)

    # stationId, stationName, mobileNo 추출
    stations = []
    for bus_station in root.findall('.//busStationList'):
        station_id = bus_station.find('stationId').text
        station_name = bus_station.find('stationName').text
        mobile_no = bus_station.find('mobileNo').text.strip() if bus_station.find('mobileNo') is not None else ''
        stations.append({'stationId': station_id, 'stationName': station_name, 'mobileNo': mobile_no})

    print(f"Results for keyword '{keyword}':")
    for station in stations:
        print(f"stationId: {station['stationId']}, stationName: {station['stationName']}, mobileNo: {station['mobileNo']}")
    print()  # 개행하여 결과 출력 구분


Results for keyword '메타폴리스(중)':
stationId: 233001219, stationName: 메타폴리스(중), mobileNo: 36667
stationId: 233001218, stationName: 메타폴리스(중), mobileNo: 36666

Results for keyword '다은마을(중)':
stationId: 233001224, stationName: 다은마을(중), mobileNo: 36664
stationId: 233001220, stationName: 다은마을(중), mobileNo: 37693

Results for keyword '예당마을.롯데캐슬(중)':
stationId: 233001282, stationName: 예당마을.롯데캐슬(중), mobileNo: 37587
stationId: 233001281, stationName: 예당마을.롯데캐슬(중), mobileNo: 36672

Results for keyword '센트럴파크뷰아파트(중)':
stationId: 233001283, stationName: 센트럴파크뷰아파트(중), mobileNo: 37590
stationId: 233001644, stationName: 센트럴파크뷰아파트(중), mobileNo: 37589

Results for keyword '개나리공원':
stationId: 233001438, stationName: 개나리공원, mobileNo: 36721
stationId: 233001437, stationName: 개나리공원, mobileNo: 37533

Results for keyword '솔빛마을.서해.쌍용아파트':
stationId: 233001450, stationName: 솔빛마을.서해.쌍용아파트, mobileNo: 36680

Results for keyword '노작마을.한화우림':
stationId: 233001487, stationName: 노작마을.한화우림, mobileNo: 37553
stationId: 233

In [89]:
dongtan = pd.read_csv('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/동탄1동 범위 내의 정류장 리스트.csv')
dongtan

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,도로명 주소,정류장ID
0,GGB233001218,메타폴리스(중),37.204267,127.067483,2023-10-16,36666,31240,경기도 화성시,경기,경기 화성시 반송동 248-1,233001218
1,GGB233001219,메타폴리스(중),37.203733,127.067517,2023-10-16,36667,31240,경기도 화성시,경기,경기 화성시 반송동 248,233001219
2,GGB233001224,다은마을(중),37.199983,127.067583,2023-10-16,36664,31240,경기도 화성시,경기,경기 화성시 반송동 180,233001224
3,GGB233001281,예당마을.롯데캐슬(중),37.210950,127.072067,2023-10-16,36672,31240,경기도 화성시,경기,경기 화성시 반송동 257,233001281
4,GGB233001283,센트럴파크뷰아파트(중),37.199433,127.064150,2023-10-16,37590,31240,경기도 화성시,경기,경기 화성시 석우동 107,233001283
5,GGB233001437,개나리공원,37.211350,127.067717,2023-10-16,37533,31240,경기도 화성시,경기,경기 화성시 반송동 109,233001437
6,GGB233001438,개나리공원,37.211700,127.067550,2023-10-16,36721,31240,경기도 화성시,경기,경기 화성시 석우동 25-9,23001438
7,GGB233001450,솔빛마을.서해.쌍용아파트,37.195817,127.069633,2023-10-16,36680,31240,경기도 화성시,경기,경기 화성시 반송동 264-3,233001450
8,GGB233001486,노작마을.한화우림,37.196183,127.077333,2023-10-16,37616,31240,경기도 화성시,경기,경기 화성시 반송동 243-4,233001486
9,GGB233002674,화성미디어센터,37.202750,127.073933,2023-10-16,55725,31240,경기도 화성시,경기,경기 화성시 반송동 243-2,233002674


# 매핑된 정류소 ID를 기반으로 정류소 경유노선 목록 조회
> 경기버스 정보_정류소 조회 서비스 api-정류소 경유노선 목록조쇠

In [107]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# API 기본 정보
api_url = "http://apis.data.go.kr/6410000/busstationservice/getBusStationViaRouteList"
service_key = "LhZifyxFqTFXCd5U9nlPliJfFyenrhzHAxtL+fIivOqevEAY6Q3hB/mZEIXViYW2bJF+b2xE5ixjrxl5aC0Zcw=="  # 실제 발급받은 키값으로 교체하세요

# 결과를 저장할 리스트
all_routes = []
keywords=list(dongtan['정류장ID'])

# 각 키워드에 대해 API 요청 및 데이터 추출
for keyword in keywords:
    params = {
        'serviceKey': service_key,
        'stationId': keyword
    }

    response = requests.get(api_url, params=params)

    # XML 파싱
    root = ET.fromstring(response.text)

    # stationId, stationName, mobileNo 추출
    for bus_station in root.findall('.//busRouteList'):
        route_id = bus_station.find('routeId').text
        route_name = bus_station.find('routeName').text
        route_type = bus_station.find('routeTypeName').text
        all_routes.append({'keyword': keyword, 'route Id': route_id, 'route Name': route_name, 'route Type': route_type})

# 데이터프레임으로 변환
df = pd.DataFrame(all_routes)

# 데이터프레임 출력
print(df)


       keyword   route Id route Name route Type
0    233001218  233000270       4108  직행좌석형시내버스
1    233001218  233000265       4403  직행좌석형시내버스
2    233001218  233000135       8501  직행좌석형시내버스
3    233001218  233000333       8502  직행좌석형시내버스
4    233001218  233000337  P9301(퇴근)  직행좌석형시내버스
..         ...        ...        ...        ...
432  233000070  233000330       H101    일반형시내버스
433  233000070  233000390        H24    일반형시내버스
434  233000070  241324005       17-1       마을버스
435  233000070  241205023         H5       마을버스
436  233000070  241205025      H6(B)       마을버스

[437 rows x 4 columns]


In [109]:
##저장
#df.to_csv('동탄1동 범위 내의 정류장 경유 노선 리스트.csv', index=False, encoding='utf-8-sig')

## 검색이 안되고 누락되는 데이터가 있지만, 감안하고 진행

In [115]:
bus_lane = pd.read_csv('C:/Users/kartboy10/Documents/GTX-A-Bus-lane-optimization/GTX_동탄역_data/동탄1동_범위 내의 정류장 경유 노선 리스트.csv')
bus_lane

,정류장,정류장ID,노선ID,노선 번호,버스유형
0,메타폴리스(중),233001218,233000270,4108,직행좌석형시내버스
1,메타폴리스(중),233001218,233000265,4403,직행좌석형시내버스
2,메타폴리스(중),233001218,233000135,8501,직행좌석형시내버스
3,메타폴리스(중),233001218,233000333,8502,직행좌석형시내버스
4,메타폴리스(중),233001218,233000337,P9301(퇴근),직행좌석형시내버스
...,...,...,...,...,...
364,동탄홈플러스,233002376,241205022,H4,마을버스
365,솔빛동탄경남아너스빌,233000070,233000330,H101,일반형시내버스
366,솔빛동탄경남아너스빌,233000070,233000390,H24,일반형시내버스
367,솔빛동탄경남아너스빌,233000070,241205023,H5,마을버스


In [116]:
## 35개가 검색되는 경유 정류장
bus_lane['정류장'].unique()

array(['메타폴리스(중)', '다은마을(중)', '예당마을.롯데캐슬(중)', '센트럴파크뷰아파트(중)', '개나리공원',
       '솔빛마을.서해.쌍용아파트', '노작마을.한화우림', '화성미디어센터', '신일유토빌정문', '동탄2동행정복지센터',
       '동탄1동행정복지센터.헌혈의집', '현대아이파크', '예당예원초', '롯데케슬정문', '미즈파크병원', '센트럴파크',
       '한화꿈에그린아파트', '메타폴리스', '예당마을', '전하리교회.우미제일', '센트럴포레스트정문', '능동주공단지',
       '동탄보건지소.우체국', '우림필유', '동탄고.동탄청소년문화의집', '서해그랑블아파트', '대우푸르지오.예당고등학교',
       '동탄119안전센터', '반월e편한세상', '잎새지하차도', 'IT단지(중)', '동양파라곤', '우미제일.전하리교회',
       '동탄홈플러스', '솔빛동탄경남아너스빌'], dtype=object)

In [118]:
lane_list = bus_lane['노선 번호'].unique()
lane_list

array(['4108', '4403', '8501', '8502', 'P9301(퇴근)', '64', '708', '9',
       '98', 'H100', 'H24', 'H65', 'H67', 'M4108', 'M4403', '12', '17',
       '81', 'H4', 'P9301(출근)', '1000', '701A', '1552', '710', '712',
       '720-3', 'H102', 'H12', '8455', '8840', '19', '19-5(B)', '6',
       '709-1', 'H2', 'H20', 'H5', 'H6(A)', '7200', '7-1A', 'H25', 'H101',
       '709', '8837', '1551', '1551B', '200', '205', '73', '27', 'H6(B)',
       '6001', '6003', '6004', '6008', '화성시광역콜버스', '116-2', '116-5',
       '203', '721', '99', 'H1', '100'], dtype=object)

In [137]:
len(lane_list)

63

# 노선별 상하차 인원 데이터와 노선 번호 데이터와 매핑

In [123]:
#평일 출퇴근 시간의 노선별 승하차 인원 데이터
sorted_passenger

,년도,월,지역,지역코드,버스노선,시점-종점 정보,시간,시간코드,주말/평일 구분,주말/평일 구분 코드,단일 승차인원,단일 하차인원,단일 총승객인원,환승 승차인원,환승 하차인원,환승 총승객인원,전체 승차인원,전체 하차인원,전체 총승객인원
0,2023,6,경기도 화성시,3124000,79,향남부영9단지-경기도종합사격장,07시~08시,7,평일,1,8,6,14,1,0,1,9,6,15
1,2023,6,경기도 화성시,3124000,79,향남부영9단지-경기도종합사격장,08시~09시,8,평일,1,5,6,11,1,2,3,6,8,14
2,2023,6,경기도 화성시,3124000,79,향남부영9단지-경기도종합사격장,18시~19시,18,평일,1,3,3,6,2,2,4,5,5,10
3,2023,6,경기도 화성시,3124000,27,경진여객차고지-두레자연중고등학교,07시~08시,7,평일,1,341,267,608,72,47,119,413,314,727
4,2023,6,경기도 화성시,3124000,27,경진여객차고지-두레자연중고등학교,08시~09시,8,평일,1,344,389,733,100,114,214,444,503,947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7117,2023,12,경기도 화성시,3124000,11월 02일,병점역.화성청년지원센터-서천2차아이파크,18시~19시,18,평일,1,229,258,487,163,140,303,392,398,790
7118,2023,12,경기도 화성시,3124000,11월 02일,병점역.화성청년지원센터-서천2차아이파크,19시~20시,19,평일,1,1,1,2,1,1,2,2,2,4
7119,2023,12,경기도 화성시,3124000,11월 02일,병점역.화성청년지원센터-서천2차아이파크,19시~20시,19,평일,1,130,152,282,122,137,259,252,289,541
7120,2023,12,경기도 화성시,3124000,11월 04일,발안만세시장-경진여객차고지,07시~08시,7,평일,1,7,0,7,2,0,2,9,0,9


In [129]:
# '버스노선'을 기준으로 필터링 + 23년 12월로 필터링
fil_df = sorted_passenger[sorted_passenger['버스노선'].isin(lane_list)]
fil_df = fil_df[fil_df['월']==12]
fil_df = fil_df.sort_values(by='버스노선')
fil_df = fil_df.reset_index(drop=True)

In [128]:
##저장
#fil_df.to_csv('동탄1동_경유 노선별 상하차 인원 데이터.csv', index=False, encoding='utf-8-sig')

In [138]:
print(fil_df['버스노선'].nunique())
print(fil_df['버스노선'].unique())

50
['100' '1000' '116-2' '116-5' '12' '1551' '1551B' '17' '19' '19-5(B)'
 '200' '203' '205' '27' '4108' '4403' '6' '6001' '6003' '6004' '6008'
 '701A' '708' '709' '709-1' '710' '712' '720-3' '7200' '721' '73' '81'
 '8501' '8502' '9' 'H1' 'H100' 'H101' 'H102' 'H12' 'H2' 'H20' 'H24' 'H25'
 'H65' 'H67' 'M4108' 'M4403' 'P9301(출근)' 'P9301(퇴근)']


* 63개의 노선 중에서, 50개의 노선에 대한 승하차 인원 데이터를 얻을 수 있음.

+ 승하차 인원이 검색되기는 하나, 출퇴근 시간 데이터가 결측치라서 sorting 과정에서 소실되는 노선도 있음.   eg. 6(B)

In [141]:
fil_df

,년도,월,지역,지역코드,버스노선,시점-종점 정보,시간,시간코드,주말/평일 구분,주말/평일 구분 코드,단일 승차인원,단일 하차인원,단일 총승객인원,환승 승차인원,환승 하차인원,환승 총승객인원,전체 승차인원,전체 하차인원,전체 총승객인원
0,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,06시~07시,6,평일,1,1,1,2,0,0,0,1,1,2
1,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,08시~09시,8,평일,1,52,61,113,0,0,0,52,61,113
2,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,18시~19시,18,평일,1,12,16,28,8,8,16,20,24,44
3,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,19시~20시,19,평일,1,3,4,7,1,1,2,4,5,9
4,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,07시~08시,7,평일,1,36,26,62,1,1,2,37,27,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,2023,12,경기도 화성시,3124000,M4403,나루마을.월드반도-신분당선강남역(중),19시~20시,19,평일,1,91,130,221,125,182,307,216,312,528
252,2023,12,경기도 화성시,3124000,P9301(출근),나루마을.월드반도-잠실광역환승센터,08시~09시,8,평일,1,0,36,36,0,7,7,0,43,43
253,2023,12,경기도 화성시,3124000,P9301(출근),나루마을.월드반도-잠실광역환승센터,07시~08시,7,평일,1,53,17,70,7,0,7,60,17,77
254,2023,12,경기도 화성시,3124000,P9301(퇴근),잠실광역환승센터-나루마을.한화꿈에그린2차,19시~20시,19,평일,1,12,35,47,0,0,0,12,35,47


# 버스 노선별 출퇴근시간의 총승객인원 데이터(승객이용 가중치)
- 또한, 승객이용 가중치 값에 대해서 Scaling 실시(100으로 나누고, 정수로 표현. 단, 0으로 나올 경우 해당 값은 1로 대체)

In [150]:
##버스 노선별 출퇴근시간의 총승객인원 데이터
df = fil_df

# Filter the dataframe for 출근시간 (06시~09시) and 퇴근시간 (18시~20시)
morning_hours = ['06시~07시', '07시~08시', '08시~09시']
evening_hours = ['18시~19시', '19시~20시']

# Group by bus route and filter by time ranges
morning_df = df[df['시간'].isin(morning_hours)]
evening_df = df[df['시간'].isin(evening_hours)]

# Calculate total passengers for each bus route in the morning and evening
morning_total = morning_df.groupby('버스노선')['전체 총승객인원'].sum().reset_index()
evening_total = evening_df.groupby('버스노선')['전체 총승객인원'].sum().reset_index()

# Rename columns for clarity
morning_total.columns = ['버스노선', '출근시간 총승객인원']
evening_total.columns = ['버스노선', '퇴근시간 총승객인원']

# Merge the results into a single DataFrame
result_df = pd.merge(morning_total, evening_total, on='버스노선', how='outer')

# Replace NaN values with 0 and convert to integer type
result_df.fillna(0, inplace=True)
result_df = result_df.astype({'출근시간 총승객인원': 'int', '퇴근시간 총승객인원': 'int'})

# Add a new column '승객이용 가중치' which is the sum of 출근시간 총승객인원 and 퇴근시간 총승객인원
result_df['승객이용 가중치'] = result_df['출근시간 총승객인원'] + result_df['퇴근시간 총승객인원']

# Scale 승객이용 가중치 and apply floor division by 100
result_df['승객이용 가중치_scaled'] = np.floor(result_df['승객이용 가중치'] / 100).astype(int)

# Replace 0 with 1 in 승객이용 가중치_scaled
result_df['승객이용 가중치_scaled'] = result_df['승객이용 가중치_scaled'].apply(lambda x: 1 if x == 0 else x)

# Sort by '승객이용 가중치' in descending order
result_df = result_df.sort_values(by='승객이용 가중치', ascending=False)

result_df

,버스노선,출근시간 총승객인원,퇴근시간 총승객인원,승객이용 가중치,승객이용 가중치_scaled
30,73,2985,2141,5126,51
13,27,2114,1590,3704,37
18,6003,1955,1363,3318,33
22,708,2083,1234,3317,33
27,720-3,1829,1263,3092,30
47,M4403,1636,1131,2767,27
8,19,1684,875,2559,25
7,17,1241,925,2166,21
46,M4108,1233,765,1998,19
40,H2,1024,877,1901,19


In [153]:
##저장
#result_df.to_csv('동탄1동_노선별 가중치.csv', index=False, encoding='utf-8-sig')

# 승객이용 가중치 데이터와 경유 정류장 데이터 매핑

In [100]:
bus_lane = pd.read_csv('./GTX_동탄역_data/동탄1동_범위 내의 정류장 경유 노선 리스트.csv')
bus_lane

,정류장,정류장ID,노선ID,노선 번호,버스유형
0,메타폴리스(중),233001218,233000270,4108,직행좌석형시내버스
1,메타폴리스(중),233001218,233000265,4403,직행좌석형시내버스
2,메타폴리스(중),233001218,233000135,8501,직행좌석형시내버스
3,메타폴리스(중),233001218,233000333,8502,직행좌석형시내버스
4,메타폴리스(중),233001218,233000337,P9301(퇴근),직행좌석형시내버스
...,...,...,...,...,...
364,동탄홈플러스,233002376,241205022,H4,마을버스
365,솔빛동탄경남아너스빌,233000070,233000330,H101,일반형시내버스
366,솔빛동탄경남아너스빌,233000070,233000390,H24,일반형시내버스
367,솔빛동탄경남아너스빌,233000070,241205023,H5,마을버스


In [118]:
bus_lane['노선 번호'].unique()

array(['4108', '4403', '8501', '8502', 'P9301(퇴근)', '64', '708', '9',
       '98', 'H100', 'H24', 'H65', 'H67', 'M4108', 'M4403', '12', '17',
       '81', 'H4', 'P9301(출근)', '1000', '701A', '1552', '710', '712',
       '720-3', 'H102', 'H12', '8455', '8840', '19', '19-5(B)', '6',
       '709-1', 'H2', 'H20', 'H5', 'H6(A)', '7200', '7-1A', 'H25', 'H101',
       '709', '8837', '1551', '1551B', '200', '205', '73', '27', 'H6(B)',
       '6001', '6003', '6004', '6008', '화성시광역콜버스', '116-2', '116-5',
       '203', '721', '99', 'H1', '100'], dtype=object)

In [152]:
# '노선 번호'와 '버스노선'을 문자열로 변환
bus_lane['노선 번호'] = bus_lane['노선 번호'].astype(str)
result_df['버스노선'] = result_df['버스노선'].astype(str)

# '승객이용 가중치_scaled' 값을 매핑하여 '가중치' 컬럼 추가
bus_lane['가중치'] = bus_lane['노선 번호'].map(result_df.set_index('버스노선')['승객이용 가중치_scaled'])

# NaN 값을 0으로 대체
bus_lane['가중치'] = bus_lane['가중치'].fillna(0).astype(int)

bus_lane

,정류장,정류장ID,노선ID,노선 번호,버스유형,가중치
0,메타폴리스(중),233001218,233000270,4108,직행좌석형시내버스,10
1,메타폴리스(중),233001218,233000265,4403,직행좌석형시내버스,12
2,메타폴리스(중),233001218,233000135,8501,직행좌석형시내버스,9
3,메타폴리스(중),233001218,233000333,8502,직행좌석형시내버스,5
4,메타폴리스(중),233001218,233000337,P9301(퇴근),직행좌석형시내버스,1
...,...,...,...,...,...,...
364,동탄홈플러스,233002376,241205022,H4,마을버스,0
365,솔빛동탄경남아너스빌,233000070,233000330,H101,일반형시내버스,2
366,솔빛동탄경남아너스빌,233000070,233000390,H24,일반형시내버스,2
367,솔빛동탄경남아너스빌,233000070,241205023,H5,마을버스,0


In [154]:
# 정류장ID를 기준으로 그룹화하고, 정류장명은 첫 번째 값을 사용하고 가중치는 합산
weight_df = bus_lane.groupby('정류장ID').agg({'정류장': 'first', '가중치': 'sum'}).reset_index()

#위/경도 좌표 추가

weight_df

,정류장ID,정류장,가중치
0,233000070,솔빛동탄경남아너스빌,4
1,233001218,메타폴리스(중),179
2,233001219,메타폴리스(중),179
3,233001224,다은마을(중),162
4,233001281,예당마을.롯데캐슬(중),180
5,233001283,센트럴파크뷰아파트(중),162
6,233001437,개나리공원,20
7,233001450,솔빛마을.서해.쌍용아파트,118
8,233001486,노작마을.한화우림,6
9,233001640,동탄2동행정복지센터,44


In [155]:
##저장
#weight_df.to_csv('동탄1동_정류장별 가중치.csv', index=False, encoding='utf-8-sig')

## 경기도 화성시 정류장 현황 데이터와 위/경도 좌표 매핑

In [162]:
## 경기도 화성시 정류장 현황 데이터와 매핑
new_stop_cord.head()

,정류장번호,정류장명,위도,경도,정보수집일,모바일단축번호,도시코드,도시명,관리도시명,도로명 주소
291,GGB233001218,메타폴리스(중),37.204267,127.067483,2023-10-16,36666,31240,경기도 화성시,경기,경기 화성시 반송동 248-1
292,GGB233001219,메타폴리스(중),37.203733,127.067517,2023-10-16,36667,31240,경기도 화성시,경기,경기 화성시 반송동 248
297,GGB233001224,다은마을(중),37.199983,127.067583,2023-10-16,36664,31240,경기도 화성시,경기,경기 화성시 반송동 180
577,GGB233001281,예당마을.롯데캐슬(중),37.210950,127.072067,2023-10-16,36672,31240,경기도 화성시,경기,경기 화성시 반송동 257
579,GGB233001283,센트럴파크뷰아파트(중),37.199433,127.064150,2023-10-16,37590,31240,경기도 화성시,경기,경기 화성시 석우동 107


In [158]:
# 데이터프레임 병합
weight_df = weight_df.merge(new_stop_cord[['정류장명', '위도', '경도']], left_on='정류장', right_on='정류장명', how='left')

# '정류장명' 컬럼은 필요없으므로 제거
weight_df = weight_df.drop(columns=['정류장명'])

In [160]:
##저장
#weight_df.to_csv('동탄1동_정류장_가중치_위경도 데이터.csv', index=False, encoding='utf-8-sig')

In [63]:
weight = pd.read_csv('./GTX_동탄역_data/동탄1동_정류장_가중치_위경도 데이터.csv')
weight

,정류장ID,정류장,가중치,위도,경도
0,233000070,솔빛동탄경남아너스빌,4,37.199117,127.074250
1,233001218,메타폴리스(중),179,37.204267,127.067483
2,233001219,메타폴리스(중),179,37.203733,127.067517
3,233001224,다은마을(중),162,37.199983,127.067583
4,233001281,예당마을.롯데캐슬(중),180,37.210950,127.072067
5,233001283,센트럴파크뷰아파트(중),162,37.199433,127.064150
6,233001437,개나리공원,20,37.211350,127.067717
7,233001450,솔빛마을.서해.쌍용아파트,118,37.195817,127.069633
8,233001486,노작마을.한화우림,6,37.196183,127.077333
9,233001640,동탄2동행정복지센터,44,37.197150,127.072600


In [92]:
#가중치가 100을 넘지 못하는 값 : 24/42
len(weight[weight['가중치']<100])

24

## 도착지 좌표 : 동탄순환대로- 37.2094185, 127.0938756

## 가중치와 위/경도 좌표로 지도 시각화

In [64]:
# 지도 생성 (초기 중심을 동탄역으로 설정)
map_center = [37.201167, 127.095111]
m = folium.Map(location=map_center, zoom_start=15, width='100%', height='100%')

# 최소 반지름과 최대 반지름 설정
min_radius = 5  # 최소 반지름
max_radius = 20  # 최대 반지름

# 가중치의 최솟값과 최댓값 계산
min_weight = weight['가중치'].min()
max_weight = weight['가중치'].max()

# 마커 추가
for _, row in weight.iterrows():
    # 가중치에 따라 반지름 계산 (선형 스케일링)
    normalized_weight = (row['가중치'] - min_weight) / (max_weight - min_weight)
    radius = min_radius + (max_radius - min_radius) * normalized_weight
    
    popup_text = f"정류장: {row['정류장']}<br>정류장ID: {row['정류장ID']}<br>가중치: {row['가중치']}"
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=radius,  # 가중치에 따라 반지름 조정
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=folium.Popup(popup_text, max_width=300),  # 팝업 설정
        tooltip=row['정류장']  # 마우스 오버시 간단 정보 표시
    ).add_to(m)
# 지도 저장
m.save('bus_stops_map.html')

m

### 반월e편한세상, 능동주공단지, 미즈파크 병원, 동탄고.동탄청소년문화의집, 우림필유

# Greedy Algorithm

In [66]:
### 최단거 + 그라디 알고리즘

# 하버사인 공식
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # 지구의 반지름, 단위: km
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# 그리디 알고리즘
def greedy_route(df, start_index, goal_lat, goal_lon):
    visited = []
    current_index = start_index
    while not df.empty:
        visited.append(df.iloc[current_index])
        df = df.drop(df.index[current_index])
        
        if df.empty:
            break
        
        current_location = visited[-1]
        current_lat = current_location['위도']
        current_lon = current_location['경도']
        
        distances = df.apply(lambda row: haversine(current_lat, current_lon, row['위도'], row['경도']), axis=1)
        df['거리'] = distances
        df = df.sort_values(by=['거리', '가중치'], ascending=[True, False]).reset_index(drop=True)
        
        # 목표 위치까지의 거리 계산
        goal_distance = haversine(current_lat, current_lon, goal_lat, goal_lon)
        
        # 목표 위치가 가장 가까운 경우 종료
        if not df.empty and goal_distance < df['거리'].iloc[0]:
            break
        
        current_index = 0
    
    return pd.DataFrame(visited)

# 시작 정류장 '동탄고.동탄청소년문화의집' (마지막 정류장)
start_index = weight[weight['정류장ID'] == 233001722].index[0]

# 목표 위치 좌표 (동탄순환대로)
goal_lat = 37.2094185
goal_lon = 127.0938756

# 경로 찾기
df_sorted = greedy_route(weight, start_index, goal_lat, goal_lon)
print(df_sorted)

        정류장ID              정류장  가중치         위도          경도        거리
27  233001722    동탄고.동탄청소년문화의집   78  37.203800  127.059467       NaN
0   233001721             우림필유   20  37.200833  127.059583  0.330044
0   233001283     센트럴파크뷰아파트(중)  162  37.199433  127.064150  0.433396
0   233001697        센트럴포레스트정문   37  37.197117  127.065750  0.294003
0   233001224          다은마을(중)  162  37.199983  127.067583  0.357728
0   233001733         서해그랑블아파트   13  37.198800  127.070267  0.271661
0   233001640       동탄2동행정복지센터   44  37.197150  127.072600  0.276356
0   233000070       솔빛동탄경남아너스빌    4  37.199117  127.074250  0.263025
0   233002674          화성미디어센터   20  37.202750  127.073933  0.404977
0   233002093            동양파라곤   36  37.204033  127.071350  0.269645
0   233002092            동양파라곤   36  37.204033  127.070983  0.032477
0   233001678            메타폴리스  130  37.205367  127.070733  0.149911
0   233001651           현대아이파크  130  37.205733  127.070617  0.042052
0   233001650  동탄1동행정복지센터.헌혈의집  14

In [70]:

####최단거리 우선 -> 실패

# 하버사인 공식
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # 지구의 반지름, 단위: km
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# 그리디 알고리즘 (최단거리 우선)
def greedy_route_shortest(df, start_index, goal_lat, goal_lon):
    visited = []
    current_index = start_index
    while not df.empty:
        visited.append(df.iloc[current_index].to_dict())
        df = df.drop(df.index[current_index])
        
        if df.empty:
            break
        
        current_location = visited[-1]
        current_lat = current_location['위도']
        current_lon = current_location['경도']
        
        # 남아있는 정류장들 중 거리만을 고려하여 다음 정류장을 선택
        distances = df.apply(lambda row: haversine(current_lat, current_lon, row['위도'], row['경도']), axis=1)
        df['거리'] = distances
        
        # 거리 기준으로 정렬
        df = df.sort_values(by='거리').reset_index(drop=True)
        
        # 목표 위치까지의 거리 계산
        goal_distance = haversine(current_lat, current_lon, goal_lat, goal_lon)
        
        # 목표 위치가 가장 가까운 경우 종료
        if not df.empty and goal_distance < df['거리'].iloc[0]:
            break
        
        current_index = 0
    
    # 목표 위치 추가
    visited.append({'정류장ID': '목표지점', '정류장': '동탄순환대로', '가중치': 0, '위도': goal_lat, '경도': goal_lon})
    
    return pd.DataFrame(visited)

# 시작 정류장 '동탄고.동탄청소년문화의집' (마지막 정류장)
start_index = weight[weight['정류장ID'] == 233001722].index[0]

# 목표 위치 좌표 (동탄순환대로)
goal_lat = 37.2094185
goal_lon = 127.0938756

# 경로 찾기
df_sorted = greedy_route_shortest(weight, start_index, goal_lat, goal_lon)
print(df_sorted)

        정류장ID              정류장  가중치         위도          경도        거리
0   233001722    동탄고.동탄청소년문화의집   78  37.203800  127.059467       NaN
1   233001721             우림필유   20  37.200833  127.059583  0.330044
2   233001283     센트럴파크뷰아파트(중)  162  37.199433  127.064150  0.433396
3   233001697        센트럴포레스트정문   37  37.197117  127.065750  0.294003
4   233001224          다은마을(중)  162  37.199983  127.067583  0.357728
5   233001733         서해그랑블아파트   13  37.198800  127.070267  0.271661
6   233001640       동탄2동행정복지센터   44  37.197150  127.072600  0.276356
7   233000070       솔빛동탄경남아너스빌    4  37.199117  127.074250  0.263025
8   233002674          화성미디어센터   20  37.202750  127.073933  0.404977
9   233002093            동양파라곤   36  37.204033  127.071350  0.269645
10  233002092            동양파라곤   36  37.204033  127.070983  0.032477
11  233001678            메타폴리스  130  37.205367  127.070733  0.149911
12  233001651           현대아이파크  130  37.205733  127.070617  0.042052
13  233001650  동탄1동행정복지센터.헌혈의집  14

In [95]:
from geopy.distance import great_circle

# 가중치가 100을 넘는 정류장만 선택하여 정렬합니다
filtered_df = weight[weight['가중치'] > 100].sort_values(by='가중치', ascending=False)

# 출발지와 목적지의 좌표를 정의합니다 (동탄고.동탄청소년문화의집에서 동탄순환대로로 가정)
start_point = (37.2038, 127.059467)
end_point = (37.2094185, 127.0938756)

# 최단거리를 구하기 위한 변수 초기화
current_point = start_point
total_distance = 0
path = []

# 탐욕 알고리즘을 사용하여 최단거리를 찾습니다
while current_point != end_point:
    min_distance = float('inf')
    next_stop = None
    
    for index, row in filtered_df.iterrows():
        stop_coords = (row['위도'], row['경도'])
        if stop_coords not in path:
            distance = great_circle(current_point, stop_coords).km
            if distance < min_distance:
                min_distance = distance
                next_stop = stop_coords
                next_stop_name = row['정류장']
    
    total_distance += min_distance
    path.append(next_stop)
    current_point = next_stop

# 최단 경로 출력
print(f"최단 거리: {total_distance:.2f} km")
print(f"경로: {start_point} -> {' -> '.join([f'{stop[0]}, {stop[1]}' for stop in path])} -> {end_point}")

# 지도 초기화
mymap = folium.Map(location=[37.2038, 127.059467], zoom_start=12)

# 출발지와 목적지 마커 추가
folium.Marker(start_point, popup='출발지', icon=folium.Icon(color='green')).add_to(mymap)
folium.Marker(end_point, popup='목적지', icon=folium.Icon(color='red')).add_to(mymap)

# 경로 추가
for stop in path:
    folium.CircleMarker(stop, radius=5, color='blue', fill=True, fill_color='blue', popup='경유지').add_to(mymap)

m

KeyboardInterrupt: 

# 운영방식

## 1. 배차간격 확인

In [109]:
#경유 노선들의 배차간격 데이터
df = pd.read_excel('./GTX_동탄역_data/공공데이터포털/공공데이터포털_경기도 화성시_마을버스노선현황_20230905.xlsx')
sorted_routes = df[df['노선번호'].isin(bus_lane['노선 번호'].unique())].sort_values(by='노선번호')
sorted_routes = sorted_routes.reset_index(drop=True)
sorted_routes['출퇴근배차'][10] = 30 ## 30~30은 30으로 대체
sorted_routes

,운수사,노선번호,기점,종점,출퇴근배차,평일배차,주말배차,주중상행첫차,주중상행막차,주말상행첫차,주말상행막차,주중하행첫차,주중하행막차,주말하행첫차,주말하행막차
0,산척여객,19-5(B),장지동농협,병점역사거리,65,65~115,65~115,05:30:00,19:10:00,05:30:00,19:10:00,07:12:00,20:52:00,07:12:00,20:52:00
1,수성여객,709-1,서동탄역,동탄역(서측),20,20~60,20~60,06:25:00,21:35:00,05:25:00,21:35:00,(회차),22:15:00,(회차),22:15:00
2,화성도시공사,H1,동탄2차고지,신동탄포레자이,25,25~30,25~30,05:20:00,22:25:00,05:20:00,22:25:00,06:20:00,23:35:00,06:20:00,23:35:00
3,산척여객,H12,장지동농협,신영통현대타운.두산위브,20,20~50,20~50,05:15:00,20:40:00,05:15:00,20:40:00,06:40:00,22:05:00,06:40:00,22:05:00
4,화성도시공사,H2,장지동농협,병점역후문,20,20~30,20~30,05:00:00,21:54:00,05:00:00,21:54:00,06:20:00,23:25:00,06:20:00,23~25
5,청진교통,H20,병점역후문,동탄역,25,25~30,65~70,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00
6,명승교통,H25,새봄초교,사랑의교회,38,38~70,70~70,06:05:00,22:05:00,06:30:00,22:15:00,06:05:00,22:10:00,06:30:00,22:15:00
7,수성여객,H4,병점역,자이파밀리에,20,20~25,40~50,22:30:00,03:10:00,22:30:00,02:50:00,22:30:00,02:40:00,22:30:00,02:50:00
8,수성여객,H5,병점역,자이파밀리에,20,20~25,40~50,22:30:00,03:15:00,22:30:00,02:55:00,22:30:00,03:15:00,22:30:00,03:15:00
9,수성여객,H6(A),서동탄역,서동탄역,30,30,60,22:30:00,02:00:00,22:30:00,01:30:00,-,-,-,-


In [110]:
sorted_routes['출퇴근배차'].mean()

28.454545454545453

> 서울 열린데이터 광장_24년5월7일 서울시버스노선기본정보 데이터

In [116]:
seoul = pd.read_excel('./GTX_동탄역_data/서울 열린데이터 광장/서울시버스노선기본정보(20240507).xlsx')
seoul

,업체명,노선번호,유형,기점명,종점명,인가대수,배차간격,거리,운행시간,최소배차,최대배차,첫차시간,막차시간
0,보광교통,0017,지선,청암동,이촌동,10,12,12.0,60.0,8,13,05:15,23:30
1,북부운수,01A,순환,예장주차장,예장주차장,16,8,16.0,60.0,6,9,06:30,23:00
2,북부운수,01B,순환,예장주차장,예장주차장,6,15,16.0,65.0,12,18,06:30,23:00
3,대원여객,0411,지선,용산차고지,AT센터.양재꽃시장,21,14,47.0,220.0,12,19,04:20,22:30
4,한성여객,100,간선,하계동,용산구청,32,10,57.0,231.0,8,13,04:00,22:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,은수교통,종로12,마을,서울대병원,종로3가,7,9,5.0,40.0,5,12,06:00,23:30
723,약수교통,종로13,마을,평창동주민센터,부암슈퍼,4,15,8.0,50.0,10,20,05:50,22:30
724,금창운수 월계점,중랑01,마을,중화1동동아약국,신이문역,2,25,4.0,24.0,25,25,06:00,23:50
725,금창운수,중랑01,마을,중화1동동아약국,신이문역,2,25,4.0,24.0,25,25,06:00,23:50


In [117]:
seoul['배차간격'].mean()

15.953232462173315

In [121]:
df = pd.read_csv('./GTX_동탄역_data/동탄1동_경유 노선별 상하차 인원 데이터.csv')
df

,년도,월,지역,지역코드,버스노선,시점-종점 정보,시간,시간코드,주말/평일 구분,주말/평일 구분 코드,단일 승차인원,단일 하차인원,단일 총승객인원,환승 승차인원,환승 하차인원,환승 총승객인원,전체 승차인원,전체 하차인원,전체 총승객인원
0,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,06시~07시,6,평일,1,1,1,2,0,0,0,1,1,2
1,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,08시~09시,8,평일,1,52,61,113,0,0,0,52,61,113
2,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,18시~19시,18,평일,1,12,16,28,8,8,16,20,24,44
3,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,19시~20시,19,평일,1,3,4,7,1,1,2,4,5,9
4,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,07시~08시,7,평일,1,36,26,62,1,1,2,37,27,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,2023,12,경기도 화성시,3124000,M4403,나루마을.월드반도-신분당선강남역(중),19시~20시,19,평일,1,91,130,221,125,182,307,216,312,528
252,2023,12,경기도 화성시,3124000,P9301(출근),나루마을.월드반도-잠실광역환승센터,08시~09시,8,평일,1,0,36,36,0,7,7,0,43,43
253,2023,12,경기도 화성시,3124000,P9301(출근),나루마을.월드반도-잠실광역환승센터,07시~08시,7,평일,1,53,17,70,7,0,7,60,17,77
254,2023,12,경기도 화성시,3124000,P9301(퇴근),잠실광역환승센터-나루마을.한화꿈에그린2차,19시~20시,19,평일,1,12,35,47,0,0,0,12,35,47


In [123]:
#출근시간
sort = df[(df['시간코드'] == 6) | (df['시간코드'] == 7) | (df['시간코드'] == 8)]
sort


,년도,월,지역,지역코드,버스노선,시점-종점 정보,시간,시간코드,주말/평일 구분,주말/평일 구분 코드,단일 승차인원,단일 하차인원,단일 총승객인원,환승 승차인원,환승 하차인원,환승 총승객인원,전체 승차인원,전체 하차인원,전체 총승객인원
0,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,06시~07시,6,평일,1,1,1,2,0,0,0,1,1,2
1,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,08시~09시,8,평일,1,52,61,113,0,0,0,52,61,113
4,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,07시~08시,7,평일,1,36,26,62,1,1,2,37,27,64
5,2023,12,경기도 화성시,3124000,1000,장외리.오리골-동탄역(동측),06시~07시,6,평일,1,5,4,9,4,4,8,9,8,17
8,2023,12,경기도 화성시,3124000,1000,장외리.오리골-동탄역(동측),07시~08시,7,평일,1,44,24,68,23,10,33,67,34,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2023,12,경기도 화성시,3124000,M4403,나루마을.월드반도-신분당선강남역(중),06시~07시,6,평일,1,231,71,302,45,7,52,276,78,354
248,2023,12,경기도 화성시,3124000,M4403,나루마을.월드반도-신분당선강남역(중),07시~08시,7,평일,1,287,236,523,71,47,118,358,283,641
249,2023,12,경기도 화성시,3124000,M4403,나루마을.월드반도-신분당선강남역(중),08시~09시,8,평일,1,244,279,523,47,71,118,291,350,641
252,2023,12,경기도 화성시,3124000,P9301(출근),나루마을.월드반도-잠실광역환승센터,08시~09시,8,평일,1,0,36,36,0,7,7,0,43,43


In [125]:
print(sort['전체 승차인원'].mean())
print(sort['전체 하차인원'].mean())

124.38709677419355
110.53548387096774


In [124]:
#출근시간
sort2 = df[(df['시간코드'] == 18) | (df['시간코드'] == 19)]
sort2


,년도,월,지역,지역코드,버스노선,시점-종점 정보,시간,시간코드,주말/평일 구분,주말/평일 구분 코드,단일 승차인원,단일 하차인원,단일 총승객인원,환승 승차인원,환승 하차인원,환승 총승객인원,전체 승차인원,전체 하차인원,전체 총승객인원
2,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,18시~19시,18,평일,1,12,16,28,8,8,16,20,24,44
3,2023,12,경기도 화성시,3124000,100,파크자이2차-파크자이2차,19시~20시,19,평일,1,3,4,7,1,1,2,4,5,9
6,2023,12,경기도 화성시,3124000,1000,장외리.오리골-동탄역(동측),19시~20시,19,평일,1,11,15,26,31,28,59,42,43,85
7,2023,12,경기도 화성시,3124000,1000,장외리.오리골-동탄역(동측),18시~19시,18,평일,1,21,29,50,3,11,14,24,40,64
13,2023,12,경기도 화성시,3124000,116-2,영통차고지(경유)-갈곶동.KCC스위첸아파트,19시~20시,19,평일,1,60,75,135,16,14,30,76,89,165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,2023,12,경기도 화성시,3124000,M4108,나루마을.월드반도-서울역버스환승센터(6번승강장)(중),19시~20시,19,평일,1,80,137,217,56,87,143,136,224,360
250,2023,12,경기도 화성시,3124000,M4403,나루마을.월드반도-신분당선강남역(중),18시~19시,18,평일,1,130,122,252,187,164,351,317,286,603
251,2023,12,경기도 화성시,3124000,M4403,나루마을.월드반도-신분당선강남역(중),19시~20시,19,평일,1,91,130,221,125,182,307,216,312,528
254,2023,12,경기도 화성시,3124000,P9301(퇴근),잠실광역환승센터-나루마을.한화꿈에그린2차,19시~20시,19,평일,1,12,35,47,0,0,0,12,35,47


In [126]:
print(sort2['전체 승차인원'].mean())
print(sort2['전체 하차인원'].mean())

113.78217821782178
126.25742574257426


## --------------------

In [3]:
df = pd.read_csv('./GTX_동탄역_data/동탄1동_노선별 가중치.csv')
df

,버스노선,출근시간 총승객인원,퇴근시간 총승객인원,승객이용 가중치,승객이용 가중치_scaled
0,73,2985,2141,5126,51
1,27,2114,1590,3704,37
2,6003,1955,1363,3318,33
3,708,2083,1234,3317,33
4,720-3,1829,1263,3092,30
5,M4403,1636,1131,2767,27
6,19,1684,875,2559,25
7,17,1241,925,2166,21
8,M4108,1233,765,1998,19
9,H2,1024,877,1901,19


In [132]:
df2 = pd.read_excel('./GTX_동탄역_data/공공데이터포털/공공데이터포털_경기도 화성시_마을버스노선현황_20230905.xlsx')
df2

,운수사,노선번호,기점,종점,출퇴근배차,평일배차,주말배차,주중상행첫차,주중상행막차,주말상행첫차,주말상행막차,주중하행첫차,주중하행막차,주말하행첫차,주말하행막차
0,금오운수,17,병점역사거리,동탄역(동측),15,15~20,25~35,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00
1,금오운수,17-1,병점역후문,동탄역,40,40~60,150~160,06:00:00,22:30:00,06:00:00,22:30:00,08:20:00,17:10:00,-,-
2,금오운수,27,병점역사거리,한림대성심병원,8,8~12,15~20,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00,06:00:00,22:30:00
3,금오운수,27-2,병점역후문,나루고교,97,97~117,-,07:43:00,16:11:00,-,-,08:33:00,17:09:00,-,-
4,다원교통주식회사,22,수현초.수현중,어천역,15,15~20,15~20,05:40:00,20:50:00,05:40:00,20:50:00,05:06:00,21:15:00,06:05:00,21:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,화성도시공사,H50-7,화성의과학대학교,사강복지회관,90,90~110,90~110,06:00:00,22:00:00,06:40:00,22:40:00,06:40:00,22:40:00,06:40:00,22:40:00
168,화성도시공사,H50-9,화성의과학대학교,화성시청서희스타힐스1단지,90,90~110,90~110,05:30:00,21:20:00,05:30:00,21:20:00,06:00:00,21:50:00,06:00:00,21:50:00
169,화성도시공사,H51,전곡항,백미리사랑방,100,100~120,100~120,06:00:00,22:00:00,06:00:00,22:00:00,06:40:00,22:40:00,06:40:00,22:40:00
170,화성도시공사,H52,전곡항,궁평유원지,110,110~140,110~140,06:00:00,21:50:00,06:00:00,21:50:00,06:45:00,22:35:00,06:45:00,22:35:00


In [135]:
# '버스노선'과 '노선번호'를 문자열로 변환
df['버스노선'] = df['버스노선'].astype(str)
df2['노선번호'] = df2['노선번호'].astype(str)


# '노선번호'와 '버스노선'을 매칭하여 '출퇴근배차' 값을 '배차간격' 컬럼에 추가
df['배차간격'] = df['버스노선'].apply(lambda x: df2[df2['노선번호'] == x]['출퇴근배차'].values[0] if x in df2['노선번호'].values else None)
df

,버스노선,출근시간 총승객인원,퇴근시간 총승객인원,승객이용 가중치,승객이용 가중치_scaled,배차간격
0,73,2985,2141,5126,51,NaN
1,27,2114,1590,3704,37,8.0
2,6003,1955,1363,3318,33,NaN
3,708,2083,1234,3317,33,NaN
4,720-3,1829,1263,3092,30,NaN
5,M4403,1636,1131,2767,27,NaN
6,19,1684,875,2559,25,21.0
7,17,1241,925,2166,21,15.0
8,M4108,1233,765,1998,19,NaN
9,H2,1024,877,1901,19,20.0


In [136]:
##저장
#df.to_csv('동탄1동_재차인원과 배차간격 데이터.csv', index=False, encoding='utf-8-sig')

## 유전 알고리즘

In [4]:
df = pd.read_csv('./GTX_동탄역_data/동탄1동_정류장_가중치_위경도 데이터.csv')
df

,정류장ID,정류장,가중치,위도,경도
0,233000070,솔빛동탄경남아너스빌,4,37.199117,127.074250
1,233001218,메타폴리스(중),179,37.204267,127.067483
2,233001219,메타폴리스(중),179,37.203733,127.067517
3,233001224,다은마을(중),162,37.199983,127.067583
4,233001281,예당마을.롯데캐슬(중),180,37.210950,127.072067
5,233001283,센트럴파크뷰아파트(중),162,37.199433,127.064150
6,233001437,개나리공원,20,37.211350,127.067717
7,233001450,솔빛마을.서해.쌍용아파트,118,37.195817,127.069633
8,233001486,노작마을.한화우림,6,37.196183,127.077333
9,233001640,동탄2동행정복지센터,44,37.197150,127.072600


In [7]:
import numpy as np
import pandas as pd
import random

# 거리 계산 함수 (유클리드 거리 사용)
def distance(stop1, stop2):
    return np.sqrt((stop1["위도"] - stop2["위도"])**2 + (stop1["경도"] - stop2["경도"])**2)

# 초기 개체군 생성
def create_initial_population(pop_size, num_stops):
    population = []
    for _ in range(pop_size):
        individual = list(np.random.permutation(num_stops))
        population.append(individual)
    return population

# 적합도 평가 함수
def evaluate_fitness(individual, bus_stops):
    fitness = 0
    for i in range(len(individual) - 1):
        fitness += distance(bus_stops.iloc[individual[i]], bus_stops.iloc[individual[i+1]])
    return fitness

# 선택 함수 (룰렛 휠 선택)
def select_parents(population, fitnesses, num_parents):
    fitness_sum = sum(fitnesses)
    probabilities = [f / fitness_sum for f in fitnesses]
    parents_indices = np.random.choice(len(population), size=num_parents, p=probabilities)
    return [population[i] for i in parents_indices]

# 교차 함수 (순서 교차)
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child1 = [None] * size
    child2 = [None] * size

    child1[start:end] = parent1[start:end]
    child2[start:end] = parent2[start:end]

    fill_positions = lambda parent, child: [
        item for item in parent if item not in child[start:end]
    ]

    child1 = fill_positions(parent2, child1)[:start] + child1[start:end] + fill_positions(parent2, child1)[start:]
    child2 = fill_positions(parent1, child2)[:start] + child2[start:end] + fill_positions(parent1, child2)[start:]

    return child1, child2

# 변이 함수 (스왑 변이)
def mutate(individual, mutation_rate):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

# 유전 알고리즘
def genetic_algorithm(bus_stops, pop_size=100, generations=1000, mutation_rate=0.01):
    num_stops = len(bus_stops)
    population = create_initial_population(pop_size, num_stops)
    
    for generation in range(generations):
        fitnesses = [evaluate_fitness(individual, bus_stops) for individual in population]
        parents = select_parents(population, fitnesses, pop_size // 2)

        next_generation = []
        for i in range(0, len(parents), 2):
            parent1, parent2 = parents[i], parents[i+1]
            child1, child2 = crossover(parent1, parent2)
            next_generation.append(mutate(child1, mutation_rate))
            next_generation.append(mutate(child2, mutation_rate))
        
        population = next_generation

    best_individual = min(population, key=lambda ind: evaluate_fitness(ind, bus_stops))
    return best_individual

# 실행
best_route = genetic_algorithm(df)

# 최적 경로 출력
best_route_stops = df.iloc[best_route].reset_index(drop=True)
print(best_route_stops)

KeyboardInterrupt: 